# Нейросетевые модели поиска. Часть II. Переранжирование.
## Домашнее задание

В этом задании вам предстоит обучить модели переранжирования с архитектурой cross-encoder и bi-encoder на датасете [VK MARCO](https://cloud.mail.ru/public/MQ3H/GVGeAWZoj). За его решение можно получить до 10 баллов, а также еще 10 бонусных за выполнение дополнительных заданий.

**Вам надо:**

- Форкнуть эту репу;
- Создать бранч, в котором вы дальше будете работать;
- Выполнить все или часть заданий ноутбука;
- Запушить ваш бранч и поставить Pull Request.

Проверяющий счекаутит вашу бранчу и проверит работу.

**Замечания:**

- При выполнении задания можете использовать код из соответствующего семинара;
- Вам предстоит сделать как минимум 2 обучения (а при выполнении дополнительных заданий и того больше), постарайтесь переиспользовать код и минимизировать копипасты;
- Старайтесь писать чистый и понятный код, не оставляйте в ноутбуке лишний мусор (выводы ячеек с pip install, ненужный закомментированный код и т.д.);
- При сдаче дз сохраните выходы ячеек с логами обучения и графиками. Если для построения графиков вы использовали tensorboard, то вставьте графики в ноутбук в виде скриншотов;
- Итоговый ноутбук должен быть работоспособен, то есть проверяющий должен иметь возможность запустить все его ячейки в том порядке, в котором они распологаются в ноутбуке, и получить результаты, аналогичные вашим.


### Данные [1 балл]

Скачайте и распакуйте датасет [VK MARCO](https://cloud.mail.ru/public/MQ3H/GVGeAWZoj). Архив содержит следующие файлы (формат аналогичен MS MARCO):
* vkmarco-docs.tsv - tsv с текстами документов;
* vkmarco-doctrain-queries.tsv - tsv с текстами запросов;
* vkmarco-doctrain-qrels.tsv - tsv с оценками релевантности запроса и документа;
* Аналогичный набор файлов для валидации.

Загрузите данные трейна и валидации в pandas.DataFrame так, чтобы он содержал следующие колонки:
* qid - id запроса;
* query - текст запроса;
* text - текст документа;
* label - оценка релевантности запроса и документа.

In [ ]:
!pip install rank_bm25

In [ ]:
import pandas as pd
import tqdm
from torchmetrics.retrieval import RetrievalMRR, RetrievalNormalizedDCG
from rank_bm25 import BM25Okapi
import torch
import transformers
from torch import nn
from torch.utils import data
from torch.utils.data import Dataset
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModel,
    XLMRobertaTokenizer,
    XLMRobertaTokenizer,
)
from sklearn.metrics import roc_auc_score

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm as tqdm_note
import gc
import time

In [ ]:
PATH = "/kaggle/input/text-reranking-competition-ir-msu-spring-2024/"

df_docs = pd.read_csv(PATH + "vkmarco-docs.tsv", sep='\t', header=None, names=['docid', 'url', 'title', 'body'])
df_queries = pd.read_csv(PATH + "vkmarco-doctrain-queries.tsv", sep='\t', header=None, names=['qid', 'query'])
df_qrels = pd.read_csv(PATH + "vkmarco-doctrain-qrels.tsv", sep=' ', header=None, names=['qid', '_', 'docid', 'label'], usecols=['qid', 'docid', 'label'])

df_train = pd.merge(df_queries, df_qrels, on='qid')
df_train = pd.merge(df_train, df_docs[['docid', 'title', 'body']], on='docid')

In [ ]:
del df_queries
del df_qrels

In [ ]:
df_train = df_train.sort_values(by='qid').reset_index(drop=True)
df_train["label"] /= max(df_train["label"])

In [ ]:
df_train.head()

,qid,query,docid,label,title,body
0,1,0 00 дом muzono net raim feat artur adil скачать,D000000009,1.000000,RaiM feat. Artur & Adil - Дом » Cкачать mp3 му...,null Показать меню Главная Узбекские песни Зар...
1,1,0 00 дом muzono net raim feat artur adil скачать,D000000008,0.666667,RaiM & Artur - Дом ft. Adil - Скачать mp3 бесп...,Скачать песню RaiM & Artur - Дом ft. Adil в mp...
2,1,0 00 дом muzono net raim feat artur adil скачать,D000000007,0.333333,Raim Feat. Artur & Adil Симпа - скачать беспла...,У нас вы можете бесплатно скачать песню Raim F...
3,1,0 00 дом muzono net raim feat artur adil скачать,D000000013,0.333333,Raim feat. Artur & Adil - Симпа - скачать бесп...,Скачать песню Raim feat. Artur & Adil - Симпа ...
4,1,0 00 дом muzono net raim feat artur adil скачать,D000000006,1.000000,Скачать RaiM feat. Artur & Adil - Дом — muzmir.kz,Тут можно скачать песню RaiM feat. Artur & Adi...


In [ ]:
df_train.to_csv("train.csv", index=False)
df_dev.to_csv("valid.csv", index=False)

### Baselines \[2 балла\]

Реализуйте подсчет метрик MRR@k и NDCG@k. Посчитайте их значения на валидационном сете для нескольких бейзлайнов:
1. Модель, делающая случайное предсказание;
1. BM25.

In [ ]:
def MRR(preds, target, qids, top_k=10):
    mrr = RetrievalMRR(top_k=top_k)
    return mrr(torch.Tensor(preds),
               torch.Tensor(target),
               indexes=torch.LongTensor(qids - min(qids)))


def NDCG(preds, target, qids, top_k=10):
    ndcg = RetrievalNormalizedDCG(top_k=top_k)
    return ndcg(torch.Tensor(preds), torch.Tensor(target), indexes=torch.LongTensor(qids - min(qids)))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
TEST_SIZE = 3_000

train_data = df_train[(TEST_SIZE < df_train["qid"])]
test_data = df_train[df_train["qid"] <= TEST_SIZE]

In [ ]:
random_preds = np.random.random(len(test_data))

print("MRR RANDOM:", MRR(random_preds, test_data['label'].values == 1, test_data['qid'].values))
print("NDCG RANDOM:", NDCG(random_preds, test_data['label'].values, test_data['qid'].values))

MRR RANDOM: tensor(0.2135)
NDCG RANDOM: tensor(0.7094)


### Создание датасетов и даталоадеров

In [ ]:
class RankDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        query, title, body, label = self.data.iloc[index, [1, 4, 5, 3]]
        return [query.lower(), (str(title) + str(body)).lower()], label

    def __len__(self):
        return len(self.data)


def compose_batch(batch):
    texts = [x for x, _ in batch]
    ys = torch.tensor([y for _, y in batch]).reshape((-1, 1)).float()
    tokens = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors="pt")
    return tokens, ys

In [ ]:
dataset_train = RankDataset(train_data)
dataset_valid = RankDataset(test_data)

In [ ]:
train_dataloader = data.DataLoader(dataset_train, shuffle=True, batch_size=256, collate_fn=compose_batch)
valid_dataloader = data.DataLoader(dataset_valid, shuffle=False, batch_size=256, collate_fn=compose_batch)

### Обучение cross-encoder \[2 балла\]

Выполните следующие задания:
* Обучите модель xlm-roberta-base в фомате cross-encoder.
* Постройте графики зависимости метрик от количества пройденных шагов.
* Посчитайте метрики для финальной модели.
* Сколько времени заняло обучение?

In [ ]:
class RankBert(nn.Module):
    def __init__(self, train_layers_count=2):
        super(RankBert, self).__init__()

        self.bert = AutoModel.from_pretrained("xlm-roberta-base")
        self.config = self.bert.config

        # freeze all layers without bias and LN
        for name, par in self.bert.named_parameters():
            if "bias" in name or "LayerNorm" in name:
                continue
            par.requires_grad = False

        layer_count = self.config.num_hidden_layers
        for i in range(train_layers_count):  # unfreeze somw layers
            for par in self.bert.encoder.layer[layer_count - 1 - i].parameters():
                par.requires_grad = True

        # map cls token emb to relevance score
        self.head = nn.Linear(self.config.hidden_size, 1)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        x = self.bert(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
        )[0][:, 0, :]  # hidden_state of [CLS]
        return self.head(x)

In [ ]:
model = RankBert(train_layers_count=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device);

In [ ]:
torch.cuda.is_available()

True

In [ ]:
class config:
    EPOCHS = 1
    LR = 1e-4
    WD = 0.01
    SAVE_DIR = "runs/cross_encоder_checkpoint"
    SAVE_INTERVAL = 1000
    BATCH_SIZE = 256
    ACCUM_BS = 1
    DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    LOG_INTERVAL = 250


writer = SummaryWriter("runs/cross_encоder_checkpoint/")

loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.LR, weight_decay=config.WD)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    pct_start=0.1,
    max_lr=config.LR,
    epochs=config.EPOCHS,
    steps_per_epoch=len(train_dataloader),
)

In [ ]:
def move_batch_to_device(batch, device):
    batch_x, y = batch
    for key in batch_x:
        batch_x[key] = batch_x[key].to(device)
    y = y.to(device)
    return batch_x, y

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.0
    running_auc = 0.0
    last_loss = 0.0

    device = config.DEVICE
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, batch in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        batch_x, y = move_batch_to_device(batch, device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(**batch_x)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, y)
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        scheduler.step()

        # Gather data and report
        running_loss += loss.item()

        y = y.cpu().int().numpy()
        if y.sum() > 0:
            # compute metric
            with torch.no_grad():
                auc = roc_auc_score(y, outputs.cpu().numpy(), labels=np.array([0, 1]))
            running_auc += np.mean(auc)
        else:
            running_auc += 1

        # logging to tb
        tb_x = epoch_index * len(train_dataloader) + i + 1
        tb_writer.add_scalar("lr", scheduler.get_last_lr()[0], tb_x)
        tb_writer.add_scalar("Train/auc", auc, tb_x)
        tb_writer.add_scalar("Train/loss", loss, tb_x)

        if i % config.LOG_INTERVAL == config.LOG_INTERVAL - 1:
            last_loss = running_loss / config.LOG_INTERVAL  # loss per batch
            last_auc = running_auc / config.LOG_INTERVAL  # loss per batch
            print("  batch {} loss: {}, auc: {}".format(i + 1, last_loss, last_auc))

            tb_writer.add_scalar("Train/running_loss", last_loss, tb_x)
            tb_writer.add_scalar("Train/running_auc", last_auc, tb_x)
            running_loss = 0.0
            running_auc = 0.0

        if i % 10 == 0:  # clean up memory
            gc.collect()
            torch.cuda.empty_cache()

        if i + 1 >= 5000:
            break

    return last_loss, last_auc

In [ ]:
config.DEVICE

device(type='cuda', index=0)

In [ ]:
epoch_number = 0

best_vloss = 1_000_000.0

for epoch in range(config.EPOCHS):
    print("EPOCH {}:".format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss, avg_auc = train_one_epoch(epoch_number, writer)

    running_vloss = 0.0
    running_vauc = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        preds = []
        for i, batch in enumerate(valid_dataloader):
            batch_x, y = move_batch_to_device(batch, config.DEVICE)
            voutputs = model(**batch_x)
            vloss = loss_fn(voutputs, y)
            running_vloss += vloss

            y = y.cpu().int().numpy()
            if y.sum() > 0:
                # compute metric
                with torch.no_grad():
                    auc = roc_auc_score(y, voutputs.cpu().numpy(), labels=np.array([0, 1]))
                running_vauc += np.mean(auc)
            else:
                running_vauc += 1

    avg_vloss = running_vloss / (i + 1)
    avg_vauc = running_vauc / (i + 1)
    print("LOSS train {} valid {}".format(avg_loss, avg_vloss))
    print("AUC train {} valid {}".format(avg_auc, avg_vauc))
    writer.add_scalar("Valid/loss", avg_vloss, epoch)
    writer.add_scalar("Valid/auc", avg_vauc, epoch)

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss

        checkpoint = {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "best_vloss": best_vloss,
        }

        torch.save(
            checkpoint, f"{config.SAVE_DIR}/ckpt_epoch_{epoch}_loss{best_vloss}.pt"
        )

    epoch_number += 1

EPOCH 1:
  batch 250 loss: 0.6886726348400116, auc: 0.6075377844617882
  batch 500 loss: 0.675019282579422, auc: 0.71487551765009
  batch 750 loss: 0.6714050917625427, auc: 0.735011804672156
  batch 1000 loss: 0.6680675461292267, auc: 0.7526184608854549
  batch 1250 loss: 0.6669481067657471, auc: 0.7530203595974533
  batch 1500 loss: 0.6659682404994964, auc: 0.7564207875266089
  batch 1750 loss: 0.6640447466373444, auc: 0.7701918903338408
  batch 2000 loss: 0.6644565877914429, auc: 0.770271971491206
  batch 2250 loss: 0.6627388315200806, auc: 0.7793648823132953
  batch 2500 loss: 0.6622070245742798, auc: 0.7798785247698724
  batch 2750 loss: 0.6608968880176544, auc: 0.7834997115391246
  batch 3000 loss: 0.6611271476745606, auc: 0.7797896867203953
  batch 3250 loss: 0.6606471478939057, auc: 0.7835831460559777
  batch 3500 loss: 0.6599810106754302, auc: 0.7829220606046728
LOSS train 0.6599810106754302 valid 0.6589420437812805
AUC train 0.7829220606046728 valid 0.7657148414442292


* Обучение модели заняло чуть более 4 часов

### Сохранение cross-encoder-модели в датасет

In [ ]:
#Сохранение модели
#torch.save(model, "cross_encoder.pth")

In [ ]:
!mkdir /kaggle/working/cross_encoder
!wget https://raw.githubusercontent.com/MaxTimoshkin/vk-msu-ir-course-spring-2024/main/homeworks/nn-reranking/dataset-metadata.json

In [ ]:
!wget https://raw.githubusercontent.com/MaxTimoshkin/vk-msu-ir-course-spring-2024/main/homeworks/nn-reranking/kaggle.json
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!mv /kaggle/working/cross_encoder.pth  /kaggle/working/cross_encoder/cross_encoder.pth
!mv /kaggle/working/dataset-metadata.json /kaggle/working/cross_encoder/dataset-metadata.json

In [ ]:
!kaggle datasets create -p /kaggle/working/cross_encoder

In [ ]:
# Загрузка модели
model = torch.load('/kaggle/input/cross-encoder/cross_encoder.pth')

### Метрики для финальной модели

In [ ]:
start_time = time.time()

def get_test_preds(model):
    model.eval()  # eval mode
    y_test = []

    for i, batch in enumerate(tqdm.tqdm(valid_dataloader, position=0, leave=True, desc=f"Test: ")):
        batch_x, y = move_batch_to_device(batch, config.DEVICE)
        with torch.no_grad():
            preds = model(**batch_x)
            y_test += [preds]

    y_test = torch.cat(y_test).view(-1).cpu().numpy()
    return y_test


y_test = get_test_preds(model)
end_time = time.time()
print(f"Test time: {end_time - start_time} seconds")

Test: 100%|██████████| 206/206 [04:50<00:00,  1.41s/it]

Test time: 290.67347955703735 seconds


In [ ]:
print(f'Test roc_auc: {roc_auc_score(dataset_valid.data["label"].values == 1, y_test, labels=np.array([0, 1]))}')

Test roc_auc: 0.7962537967116008


In [ ]:
k = 10
mrr = MRR(y_test, test_data["label"].values == 1, test_data["qid"].values, k)
ndcg = NDCG(y_test, test_data["label"].values, test_data["qid"].values, k)
print(f"MRR@{k}: {mrr}, NDCG@{k}: {ndcg}")

MRR@10: 0.3191421329975128, NDCG@10: 0.770759105682373


### Отправка решения

In [ ]:
submission = pd.read_csv('/kaggle/input/text-reranking-competition-ir-msu-spring-2024/sample_submission.csv')
submission

,QueryId,DocumentId
0,3,D000000029
1,3,D000000040
2,3,D000000039
3,3,D000000035
4,3,D000000034
...,...,...
152687,42768,D001258585
152688,42768,D001258581
152689,42768,D001258575
152690,42768,D001258584


In [ ]:
test = pd.read_csv('/kaggle/input/text-reranking-competition-ir-msu-spring-2024/vkmarco-doceval-queries.tsv',
                   sep='\t', header=None, names=['QueryId', 'query'])
test

,QueryId,query
0,7815,куркумин эвалар побочные действия
1,26784,стрижки на волнистых волосах средней длины 2019
2,39924,1 бальбоа 1941
3,42626,установка передних брызговиков фольксваген тра...
4,39505,музыка мія бойко 10 в 1
...,...,...
4995,1385,sokolov jewelry википедия
4996,2271,байкал подлеморье энхалук
4997,26304,не найден подписанный драйвер
4998,9323,незаряженный конденсатор емкость 100мкф


In [ ]:
df_docs.rename(columns = {'docid': 'DocumentId'}, inplace=True)
df_docs

,DocumentId,url,title,body
0,D000000001,zvuk.top/tracks/%D0%B4%D0%BE%D0%BC-raim-feat-a...,Дом raim feat artur adil музыка в MP3 🎵 – скач...,Скачивай и слушай 🎧 raim дом o2 2018 на Zvuk.t...
1,D000000002,musickz.kz/index.php?newsid=3221,Скачать RaiM feat. Artur & Adil - Дом 2018,Скачать песню RaiM feat. Artur & Adil - Дом бе...
2,D000000003,muzvuk.net/1965-raim-ft-artur-amp-adil-dom.html,RaiM ft. Artur & Adil - Дом - Скачать mp3,Скачать песню RaiM ft. Artur & Adil - Дом в Фо...
3,D000000004,zvuq.net/4415-raim-feat-artur-adil-dom.html,RaiM feat. Artur & Adil - Дом - скачать в mp3 ...,Скачать RaiM feat. Artur & Adil - Дом в mp3 фо...
4,D000000005,mussic.kz/kazahskie-pesni/2417-raim-feat-artur...,Скачать Дом - RaiM feat. Artur & Adil — Mussic.kz,Здесь можете бесплатно скачать Дом - RaiM feat...
...,...,...,...,...
1258651,D001258652,hy.wikipedia.org/wiki/%D5%84%D5%A1%D5%B5%D5%AB...,Մայիսի 9 - Վիքիպեդիա՝ ազատ հանրագիտարան,null Մայիսի 9 << Մայիս >> Կի Եկ Եք Չո Հի Ու Շա...
1258652,D001258653,ysu.am/science/ru/Lusine-Grigoryan/type/1/show...,NaN,null Հայերեն English 02 Сентября 2022 | Пятниц...
1258653,D001258654,ac.qq.com/Comic/comicInfo/id/637247,大英雄的女友超级凶-大英雄的女友超级凶在线漫画-在线漫画-腾讯动漫官方网站,系统漫画《大英雄的女友超级凶》，简介：根据阿里巴巴文学二次元签约作者大仙的同名小说改编。他是...
1258654,D001258655,vk.com/video357967041_456239061,善良的女祕書 [中文字幕].. — Видео | ВКонтакте,Смотрите онлайн 善良的女祕書 [中文字幕].. 20 мин 14 с. В...


In [ ]:
test_data = pd.merge(submission, test, on='QueryId')
test_data = pd.merge(df_docs, test_data, on='DocumentId')
test_data = test_data.drop(columns='url')
test_data

,DocumentId,title,body,QueryId,query
0,D000000029,Текила Максимо де Кодорниз Сильвер | Федеральн...,Федеральный реестр алкогольной продукции онлай...,3,0 5 текила максимо де кодорниз сильвер 38
1,D000000030,"Текила. Ром. Джин - Алкоголь, энергетики - Руч...","Текила. Ром. Джин, Алкоголь, энергетики, Интер...",3,0 5 текила максимо де кодорниз сильвер 38
2,D000000031,Текила Jose Cuervo Especial Silver 0.7 л 38% (...,Почитать отзывы о товаре 👉 Текила Jose Cuervo ...,3,0 5 текила максимо де кодорниз сильвер 38
3,D000000032,"Текила ""Максимо Де Кодорниз Сильвер"" 0,5л. 38%",SCID scidФирменный магазин Московского Завода ...,3,0 5 текила максимо де кодорниз сильвер 38
4,D000000033,Меню и цены - Чайхана Чайхауз в Домодедово в М...,⭐⭐⭐⭐⭐Представляем вашему вниманию цены на разл...,3,0 5 текила максимо де кодорниз сильвер 38
...,...,...,...,...,...
152687,D001258581,Հայկական ծագում ունեցող անձնանունների ցանկ - Վ...,null Հայկական ծագում ունեցող անձնանունների ցան...,42768,ապօրինի ծագում ունեցող
152688,D001258582,Դիտարկումներ «Ապօրինի ծագում ունեցող գույքի բռ...,ՀՀ արդարադատության նախարարության կողմից 2019 թ...,42768,ապօրինի ծագում ունեցող
152689,D001258583,1or.am | «Հրապարակ». Երկուշաբթի Գլխավոր դատախա...,null ﻿ Հրահանգի կատարման լկտիության համար՝ պար...,42768,ապօրինի ծագում ունեցող
152690,D001258584,ArmDaily.am | Արդարադատության նախարարությունու...,"null ﻿ Երևան, 16.Հուլիս.2022, 00 : 00 Խմբագրակ...",42768,ապօրինի ծագում ունեցող


In [ ]:
class TestDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        query, title, body = self.data.iloc[index, [4, 1, 2]]
        return [query.lower(), (str(title) + str(body)).lower()]

    def __len__(self):
        return len(self.data)

def compose_batch(batch):
    return tokenizer(batch, padding=True, truncation=True, max_length=64, return_tensors="pt")

def move_x_batch_to_device(batch_x, device):
    for key in batch_x:
        batch_x[key] = batch_x[key].to(device)
    return batch_x

In [ ]:
test_ds = TestDataset(test_data)
test_dl = data.DataLoader(test_ds, shuffle=False, batch_size=256, collate_fn=compose_batch)

In [ ]:
def get_test_preds(model):
    model.eval()
    y_test = []

    for i, batch_x in enumerate(tqdm.tqdm(test_dl, position=0, leave=True, desc=f"Test: ")):
        batch_x = move_x_batch_to_device(batch_x, config.DEVICE)

        with torch.no_grad():
            y_test += [model(**batch_x)]

    return torch.cat(y_test).view(-1).cpu().numpy()

y_test = get_test_preds(model)

Test: 100%|██████████| 597/597 [15:38<00:00,  1.57s/it]


In [ ]:
test_data['score'] = y_test
test_data

,DocumentId,title,body,QueryId,query,score
0,D000000029,Текила Максимо де Кодорниз Сильвер | Федеральн...,Федеральный реестр алкогольной продукции онлай...,3,0 5 текила максимо де кодорниз сильвер 38,-0.097398
1,D000000030,"Текила. Ром. Джин - Алкоголь, энергетики - Руч...","Текила. Ром. Джин, Алкоголь, энергетики, Интер...",3,0 5 текила максимо де кодорниз сильвер 38,-0.346297
2,D000000031,Текила Jose Cuervo Especial Silver 0.7 л 38% (...,Почитать отзывы о товаре 👉 Текила Jose Cuervo ...,3,0 5 текила максимо де кодорниз сильвер 38,-0.376945
3,D000000032,"Текила ""Максимо Де Кодорниз Сильвер"" 0,5л. 38%",SCID scidФирменный магазин Московского Завода ...,3,0 5 текила максимо де кодорниз сильвер 38,-0.146997
4,D000000033,Меню и цены - Чайхана Чайхауз в Домодедово в М...,⭐⭐⭐⭐⭐Представляем вашему вниманию цены на разл...,3,0 5 текила максимо де кодорниз сильвер 38,-0.384289
...,...,...,...,...,...,...
152687,D001258581,Հայկական ծագում ունեցող անձնանունների ցանկ - Վ...,null Հայկական ծագում ունեցող անձնանունների ցան...,42768,ապօրինի ծագում ունեցող,-2.853930
152688,D001258582,Դիտարկումներ «Ապօրինի ծագում ունեցող գույքի բռ...,ՀՀ արդարադատության նախարարության կողմից 2019 թ...,42768,ապօրինի ծագում ունեցող,-2.320546
152689,D001258583,1or.am | «Հրապարակ». Երկուշաբթի Գլխավոր դատախա...,null ﻿ Հրահանգի կատարման լկտիության համար՝ պար...,42768,ապօրինի ծագում ունեցող,-2.477823
152690,D001258584,ArmDaily.am | Արդարադատության նախարարությունու...,"null ﻿ Երևան, 16.Հուլիս.2022, 00 : 00 Խմբագրակ...",42768,ապօրինի ծագում ունեցող,-2.211691


In [ ]:
sorted_df  = test_data.sort_values(by=['QueryId', 'score'], ascending=[True, False])
sorted_df

,DocumentId,title,body,QueryId,query,score
0,D000000029,Текила Максимо де Кодорниз Сильвер | Федеральн...,Федеральный реестр алкогольной продукции онлай...,3,0 5 текила максимо де кодорниз сильвер 38,-0.097398
3,D000000032,"Текила ""Максимо Де Кодорниз Сильвер"" 0,5л. 38%",SCID scidФирменный магазин Московского Завода ...,3,0 5 текила максимо де кодорниз сильвер 38,-0.146997
6,D000000035,Текила – виды и производители мексиканской водки,"Текила – модный алкогольный напиток, ставший в...",3,0 5 текила максимо де кодорниз сильвер 38,-0.231669
9,D000000038,"Текила Sombrero Silver: отзывы, описание, прои...",На данный момент алкогольный продукт выпускают...,3,0 5 текила максимо де кодорниз сильвер 38,-0.247382
7,D000000036,Текила Tequila Don Cortez Silver 0.75 л купить...,Купить Текила Tequila Don Cortez Silver 0.75 л...,3,0 5 текила максимо де кодорниз сильвер 38,-0.263868
...,...,...,...,...,...,...
152676,D001258570,Verelq News | Ապօրինի ծագում ունեցող գույքի բռ...,Ապօրինի ծագում ունեցող գույքի բռնագանձման գործ...,42768,ապօրինի ծագում ունեցող,-2.512634
152691,D001258585,Ինչպե՞ս բռնագանձել ապօրինի ծագում ունեցող գույ...,"null Wednesday, 13 07 2022 ՀԱՅԱՍՏԱՆ ՔԱՂԱՔԱԿԱՆ ...",42768,ապօրինի ծագում ունեցող,-2.529945
152683,D001258577,ՀԱՅԱՍՏԱՆԻ ՀԱՆՐԱՊԵՏՈՒԹՅԱՆ ՔՐԵԱԿԱՆ ԴԱՏԱՎԱՐՈՒԹՅԱՆ...,ՀԱՅԱՍՏԱՆԻ ՀԱՆՐԱՊԵՏՈՒԹՅԱՆ ՔՐԵԱԿԱՆ ԴԱՏԱՎԱՐՈՒԹՅԱՆ...,42768,ապօրինի ծագում ունեցող,-2.618495
152686,D001258580,Ապօրինի գույքի բռնագանձման մասին օրենքը հակասա...,null ՓԱՍՏԱՉԱՓ ՍԽԱԼ ՄԵԾԱՄԱՍԱՄԲ ՍԽԱԼ ԱՌԱՆՑ ՎՃՌԻ ...,42768,ապօրինի ծագում ունեցող,-2.837952


In [ ]:
sorted_df[['QueryId', 'DocumentId']].to_csv('submission.csv', index=False)